# 对numpy矩阵的保存和读取

In [13]:
import numpy as np

# 点云地图点数
point_size = 160
# 训练集大小 暂设 100
train_size = 100
# 最低深度
min_depth = -5 
# 最高深度
max_depth = 10
np_save = np.ones((train_size,point_size,point_size))
for i in range(0,train_size):
    # 生成随机地图
    np_save[i,:,:] = np.random.randint(min_depth, max_depth, (point_size,point_size))

# np.save("test_x",np_save)
goal = [
    [0,0],[0,50],[50,0],[50,50],
    [4]
    ]
goal1 = [
    [0,0],[0,2],[2,0],[2,2],
    [4]
]

### 另一种保存的方式
j = np.array([1, 5, 8])
k = np.array([11, 31, 9])
l = np.array([67, 40, 23])
 用默认key值
np.savez("arr3",  j, k, l)
 要写出完整的文件名，包括后缀
arr3 = np.load("arr3.npz")
print(arr3.files)
print(arr3["arr_0"])


## 读取

In [14]:
train_x = np.load("test_x.npy")
print(train_x.shape)

(100, 160, 160)


## 参数设置 
生成（100，32，32）的tile_map(训练集)

In [50]:
import numpy as np

# 任务深度
task_depth = 6
# 单元格 长和宽
cell_x = 5
cell_y = 5
# tile 索引
index_x = 0
index_y = 0
# 将实际地图信息转化为0，1二进制信息
tile_space = np.where(np_save > task_depth,1,0)
# np.sum(tile_space[0:cell_x,0:cell_y])
tile_grid = np.zeros([train_size ,int(point_size/cell_x),int(point_size/cell_y)],dtype = int)
summ = 0
# 将tile_grid按照一定规则转换
for t in range(0,train_size):
    for i in range(0,len(tile_grid[0])):
        for j in range(0,len(tile_grid[1])):
            tile_grid[t,i,j] = 1 if np.sum(tile_space[t,i * cell_x:i * cell_x + cell_x,j * cell_y:j*cell_y+cell_y]) > 0 else 0 
            if tile_grid[t,i,j] == 0:
                summ = summ + 1
np.save("test_x",tile_grid)
print(summ)

426


In [ ]:
# 输出动作 [挖掘，上，下，左，右]
import copy


action_space_exc = [1,0,0,0,0]
action_space_up = [0,1,0,0,0]
action_space_down = [0,0,1,0,0]
action_space_left = [0,0,0,1,0]
action_space_right = [0,0,0,0,1]
action_list_space = []

length_x = len(tile_grid[0])
length_y = len(tile_grid[1])

# TEM测试集
for i in range(0,train_size):
    index_x = 0
    index_y = 0
    action_list = []
    direct_bool = True
    while index_x < len(tile_grid[0]) and index_y < len(tile_grid[1]):
        if tile_grid[i,index_x,index_y] != 0:
            action_list.append(action_space_exc)
        if direct_bool:
            if index_x < length_y - 1:
                action_list.append(action_space_right)
                index_x = index_x + 1
            else:
                action_list.append(action_space_down)
                index_y = index_y + 1
                direct_bool = bool(1-direct_bool)
        else:
            if index_x > 0:
                action_list.append(action_space_left)
                index_x = index_x - 1
            else:
                action_list.append(action_space_down)
                index_y = index_y + 1
                direct_bool = bool(1-direct_bool)
    action_list_space.append(copy.deepcopy(action_list))

print(action_list_space[0])

## 下面是CEM的测试集